In [3]:
import cycpd
import matplotlib.pyplot as plt
import pycpd
import numpy as np
import os
import glob
import shutil
import open3d as o3d
import time
import h5py
import pandas as pd
import utils

### Gaussian Process & Multivariate Normal Distribution

In [5]:
paramsFiles = glob.glob(os.path.join(r"./data/params", "sRtParams_*.hdf5"))
toothIndices = [int(os.path.basename(f).split(".")[0].split("_")[-1]) for f in paramsFiles]

In [6]:
dfParam = pd.DataFrame(columns=["tag"])
for f,tID in zip(paramsFiles,toothIndices):
    tags, scales, rotMats, transVecs = utils.readRegistrationParamsFromHDF5(h5File=f, toothIndex=tID)
    rotAngles = utils.getRotAngles(rotMats)
    rx, ry, rz = np.hsplit(rotAngles, 3)
    tx, ty, tz = np.hsplit(transVecs, 3)
    tags = [tag[:-1] for tag in tags]
    dataTemp = {"tag":tags, 
                "{}s".format(tID):scales, 
                "{}rx".format(tID):rx.flatten(), 
                "{}ry".format(tID):ry.flatten(), 
                "{}rz".format(tID):rz.flatten(),
                "{}tx".format(tID):tx.flatten(), 
                "{}ty".format(tID):ty.flatten(), 
                "{}tz".format(tID):tz.flatten()} 
    dfTemp = pd.DataFrame(dataTemp)
    dfParam = dfParam.merge(dfTemp, how="outer", on="tag")

In [9]:
dfParam

,tag,11s,11rx,11ry,11rz,11tx,11ty,11tz,12s,12rx,...,46tx,46ty,46tz,47s,47rx,47ry,47rz,47tx,47ty,47tz
0,65,1.575657,-0.417926,-0.481351,0.249453,10.117639,-2.097857,0.277476,1.218601,-0.328558,...,1.302736,-6.269171,-1.038112,1.040174,0.015166,0.143515,-0.531078,-0.859560,-13.322393,-5.262710
1,87,1.221864,0.217351,-0.037992,-0.276693,2.923883,2.645940,-4.395098,1.026329,0.047310,...,-1.715662,-2.563255,2.736868,0.936835,0.069393,-0.156241,-0.084532,-5.121869,-2.473417,2.940782
2,84,1.243813,-0.291333,0.464359,-0.150202,-10.057017,-3.592691,-1.126615,1.034752,-0.298091,...,-2.753845,-8.467804,-0.974047,0.942282,0.179305,0.160814,-0.623633,-5.942495,-16.496163,-6.611356
3,4,1.167750,0.069135,-0.140152,-0.037460,3.247426,1.080696,-3.003360,1.189040,0.043493,...,1.429975,1.252429,-1.776484,1.135607,0.011708,0.021838,0.012897,2.474693,0.062724,1.736226
4,111,1.171392,-0.324573,-0.445354,-0.031206,6.563996,-5.149365,3.251975,1.132319,-0.206623,...,5.772693,1.391698,-1.303359,1.251403,-0.112454,-0.020584,-0.066664,7.595420,0.337184,6.086091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,72,0.869813,-0.021190,0.176258,-0.150886,-2.404517,-0.221881,1.556239,0.888826,-0.053049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.097345,0.105345,...,0.884411,-1.768931,-1.862404,1.017284,-0.012574,-0.054611,-0.080661,-0.512782,-1.626821,1.140145
127,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.953832,0.108155,...,-0.459578,0.453675,-1.427462,1.021983,-0.056423,-0.056087,0.381929,-5.032038,9.070757,3.986438
128,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.877819,-0.056448,...,-0.539552,-1.343462,0.074190,0.964833,-0.033392,0.154054,-0.086390,1.969761,-0.782816,-4.397893


In [79]:
A = dfParam.iloc[:,1:8].to_numpy()
covA = np.ma.cov(np.ma.masked_invalid(A), rowvar=False)
assert not covA.mask.any() #检查是否有nan
covMat = covA.data

In [80]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [81]:
is_pos_def(covMat)

True

In [23]:
pd.DataFrame(covMat).to_csv("covMat.csv",index=False,header=False)

In [7]:
variances = np.ma.var(np.ma.masked_invalid(A), axis=0, ddof=1)
assert not variances.mask.any() #检查是否有nan
std = np.sqrt(variances.data)

In [8]:
rhoCoef = covMat / np.multiply(std[:,None],std)
assert (np.abs(rhoCoef.diagonal()-1.)<1e-6).all() #对角线元素全是1

In [9]:
largeRhoCoef = rhoCoef.copy()
largeRhoCoef[np.abs(largeRhoCoef)<0.6] = 0.
pd.DataFrame(largeRhoCoef).round(2).to_csv("rhoCoef.csv",index=False,header=False)